In [ ]:
import torch
from pathlib import Path
import os
os.chdir('../../ours_pipeline_SDv1.5')
from metrics_utils import VGGCalculator, CLIPScorer, calculate_aesthetic_score, DINOStyleScorer
from notebooks.prompts_dict import common_prompts as prompts

In [3]:
def read_yaml(yaml_path):
    import yaml
    with open(yaml_path, 'r') as f:
        data = yaml.safe_load(f)  # 使用 safe_load 避免安全风险
        return data
        

In [33]:
import numpy as np


VGG_Scorer = VGGCalculator()
CLIP_Scorer = CLIPScorer()
output_path = "/userhome/wangjunke/M3S/metric_SDv1.5/output_final"
test_style_path = "style_assign_4/a cat/multi_style"
yaml_path = os.path.join(output_path, test_style_path, "config.yaml")
full_image_path = os.path.join(output_path, test_style_path, "out_transfer---seed_42.png")
base_style_image_path = "/userhome/wangjunke/M3S/metric_SDv1.5"
vgg_values = []
aesthetic_score = 0
clip_score = 0

In [34]:
yaml_data = read_yaml(yaml_path)
multi_style_image_path =  yaml_data['style_image_paths']
for i in range(len(yaml_data['style_image_paths'])):
    path = yaml_data['style_image_paths'][i]
    multi_style_image_path[i] = os.path.join(base_style_image_path, path)
style_weight = yaml_data['style_weights']
prompt = yaml_data['prompt']
print(multi_style_image_path, style_weight)
print(prompt)

['/userhome/wangjunke/M3S/metric_SDv1.5/style_base/4.png'] [1.0]
a sketch of a cat


In [35]:
clip_score += CLIP_Scorer(full_image_path, prompt)
# for style_image_path in multi_style_image_path:
#     vgg_loss += VGG_Scorer.calculate_similarity(full_image_path, style_image_path)
for style_path in multi_style_image_path:
    v = VGG_Scorer.calculate_similarity(full_image_path, style_path).item()   # scalar tensor -> float
    vgg_values.append(v)
aesthetic_score += float(calculate_aesthetic_score(full_image_path))
print("CLIP Score:", clip_score)
print("VGG Loss:", vgg_values)
print("style_weight:", style_weight)
print("Aesthetic Score:", aesthetic_score)

CLIP Score: 0.3173828125
VGG Loss: [0.008660847321152687]
style_weight: [1.0]
Aesthetic Score: 4.830204486846924
